Given timestamps of logins, figure out how many people on Facebook were active all
seven days of a week on a mobile phone?

In [1]:
import pandas as pd
import numpy as np

In [2]:
#First I will sort so that we are only looking at Mobile phone users

In [ ]:
mobile_data = data[data['interface'] == 'Mobile'].copy()

In [ ]:
#Convert timestamps to days

In [ ]:
mobile_data['day'] = mobile_data['login'].resample('D')

In [ ]:
#Sort for last 7 days

In [ ]:
last_week = mobile_data[mobile_data['day'] > pd.to_datetime('2018-03-13')]

In [ ]:
filtered_week = last_week.drop(['login'], axis=1)

In [ ]:
filtered_week.drop_duplicates(inplace=True)

In [ ]:
counts = filtered_week.groupby('user').count()

In [ ]:
all_seven = counts[counts['day'] == 7]['user']

How do you determine what product in Facebook was used most by the non-employee
users for the last quarter?

In [3]:
#Assuming that a count of users will be sufficient. Other options include a count of logins

In [ ]:
#First, lets sort for all users that are not employees
non_ee = data[data['EE'] == False].copy()

In [ ]:
#Now we can group by product
products = non_ee.groupby('Product').count() #This allows us to count logins. Could use mean login time, etc.

In [ ]:
products.sort_values(['logins'], ascending = False)[0]

Merging some stuff

In [38]:
user_usage = pd.read_csv('https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/user_usage.csv')
user_device = pd.read_csv('https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/user_device.csv')
android_devices = pd.read_csv('https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/android_devices.csv')

In [39]:
user_usage.head(2)

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788


In [45]:
user_device.head(2)

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3


In [46]:
android_devices.head(2)

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21


In [56]:
merged = user_usage.merge(user_device[['use_id', 'device', 'platform']], on='use_id', how='left')

In [57]:
merged.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,device,platform
0,21.97,4.82,1557.33,22787,GT-I9505,android
1,1710.08,136.88,7267.55,22788,SM-G930F,android
2,1710.08,136.88,7267.55,22789,SM-G930F,android
3,94.46,35.17,519.12,22790,D2303,android
4,71.59,79.26,1557.33,22792,SM-G361F,android


In [63]:
final_merge = merged.merge(android_devices[['Retail Branding', 'Model']], left_on='device', right_on='Model', how='left')

In [65]:
final_merge.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,device,platform,Retail Branding,Model
0,21.97,4.82,1557.33,22787,GT-I9505,android,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,SM-G930F,android,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,SM-G930F,android,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,D2303,android,Sony,D2303
4,71.59,79.26,1557.33,22792,SM-G361F,android,Samsung,SM-G361F


In [68]:
final_merge.groupby('Retail Branding').mean()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
Retail Branding,,,,
HTC,299.842955,93.059318,5144.077955,22893.431818
Huawei,81.526667,9.500000,1561.226667,22974.333333
LGE,111.530000,12.760000,1557.330000,22907.000000
Lava,60.650000,261.900000,12458.670000,23004.000000
Lenovo,215.920000,12.930000,1557.330000,22974.000000
Motorola,95.127500,65.666250,3946.500000,22999.437500
OnePlus,354.855000,48.330000,6575.410000,22937.166667
Samsung,191.010093,92.390463,4017.318889,22922.944444
Sony,177.315625,40.176250,3212.000625,22905.125000


# Sample Questions

Given a table of friend requests sent and friend requests received, find the user with the most friends.

sender | target | date
approver | source | date

#### My first thoughts are that sender and source are the same field whereas target and approver are the same fields. I would want to make sure that if I join these tables that I make sure to join on the pair of values as the sender could have multiple sent requests, but they would need to be mapped to the appropriate approvers, and visa versa. 

#### Also, on table2, the approver and the source both get a friend from this interaction.

##### I'm assuming there are no duplicates and no unfriendings.

#### So I believe that I don't actually need to join these tables. Instead I can take a look at all of the users located in the approver field and all of the users in the source field and, using .value_counts() can find the user that appears the most.

all_friends = table2['approver'] + table2['source]
values = all_friends.value_counts(ascending=False)
values[0]

how to detect the fake information input from the user

#### First, lets determine what we mean by fake information. Are we talking about fraudulent accounts? 
One way to determine this would be to compare the user's information to other similar users. For example, if a user is from Everett, WA and they specify that they went to Woody's HighSchool, we can take a look at other people in Everett and find that this high-school isn't listed in the area. 

Also, fraudulent accounts are typically associated with lost of posting, but not a lot of enagegment, or engagement only in a specific area (e.g. Russian politics, etc.). Comparing normal post to comment ratios across certain facebook demographics to the users in question could raise red flags. 

Users that are created recently that do not have a lot of friend requests sent to them from older users could also be a red flag as I'm assuming that a lot of fake users will be newly created.

A flag-feature could also be useful but could come with some drawbacks.

Checking for values that are clearly out of bounds (age > 110, etc.).

Comparing to lists of possible values (state abbreviations, etc)

Likes/user and minutes spent on a platform are increasing but total number of users is decreasing. What could be causing this?

 SQL Given tables: employees(id, unixname,team,role,days_since_started) projects(id,name,....) commits(id,file_path,proj_id,auth_id,timestamp) Find the number of unique employees per project per month?

employees: id | username | team | role | days_since_started
projects:  id | name
commits:   id | file_path | proj_id | auth_id | timestamp

#### The first thing that I'm going to do is merge the projects and the commits tables. These can be merged on the projects-id field and the commits-proj_id field. I believe that an inner join makes the most sense given that if a project wasn't worked on during the month, it would not appears in the commits table. As the commits table is my only link from projects to employees, it will not capture employees that were part of projects that had no commits during the time period specified by the timestamp field.
project_info = projects.merge(commits, left_on='id', right_on='proj_id', how='inner')
#### My next step will be to merge the project_info table with the employees table so that each row will now list the employee information as well as the project information for each commit. Once again, this will be an inner join, as if a user wasn't part of a commit, I do not have a way to map them to a projects. 
merged = project_info.merge(employees, left_on='commits_id', right_on='username', how='inner')
#### To break the timestamps down into months, I will add a month column.
merged['month'] = merged['timestamp'].month()
#### Users are going to be listed multiple times for projects if they made multiple commits during a month. As such, i will drop all duplicates for which the username, project id, and month match.
merged.drop_duplicates(['username', 'proj_id', 'month'], inplace=True)
#### I now have a dataframe of all the unique users, projects, and months. I can group by the month and proj_id, using the count() method to determine the number of unique users that fall into each group.
monthly_projects = merged.groupby(['month', 'proj_id']).count()
#### Outputting the username column from this dataframe will then given me the month, the project, and the number of unique users per project.
monthly_projects['username']

How do you calculate monthly active users, churned users and resurrected users from a user activity log with userID and DateTime

activity: userID | DateTime

#### Active users is simple enough. That would be the number of users that were active within the last 30 days. I can filter for that and then count the number of unique users. I'm assuming that the DateTime column is in datetime format.
last_month = activity[activity['DateTime'] > pd.to_datetime('2018-02-20')]
unique_last_month = last_month['userID'].unique()
unique_last_month.shape[0]

#### Churned users will be a bit more difficult as this will be users that were active 2 months ago but were not during the last month. First I will need a list of unique users that were active two months ago.
prior_users = activity[(activity['DateTime] > pd.to_datetime('2018-01-20')) & (activity['DateTime'] <      pd.to_datetime('2018-02-20'])]
unique_prior_users = prior_users['userID'].unique()
#### Now I have to compare this series with the series that I made previously to see which users dropped off.
unique_prior_users.isin(unique_last_month).sum()

#### Resurrected users will be the opposite of this - users that are new last month, but were not active two months ago.
-(unique_last_month.isin(unique_prior_users)).sum()

How many high schools that people have listed on their profiles are real? How do we find out, and deploy at scale, a way of finding invalid schools?

Facebook sees that likes are up 10% year over year, why could this be?



If a PM says that they want to double the number of ads in Newsfeed, how would you figure out if this is a good idea or not?

We have two options for serving ads within Newsfeed: 1 - out of every 25 stories, one will be an ad 2 - every story has a 4% chance of being an ad For each option, what is the expected number of ads shown in 100 news stories? If we go with option 2, what is the chance a user will be shown only a single ad in 100 stories? What about no ads at all?

#### The expected value will be the same for both in 100 stores as 100/25 = 4 and 0.4*100 = 4.
#### The chance that a user will be shown only a single ad in 100 stores can be modeled best with a binomial distribution. This models the number of successes (ads shown) in n events (100 news stories). My first thought is that it's probably very rare, given that the mean of the binomial distribution is going to be four so one news story is going to lie a number of standard deviations away from the mean to the left.

#### What I'm looking for in this case is the probability of a user seeing 1 ad in 100 stories with a mean add rate of 4 per 100. 
#### P(1 ad | mean rate of 4/100)

#### In python this could be done by creating a binomial model with scikitlearn binom(100, 0.04) and then calling the cdf on it to find the p-value.  >>>>>    binom.cdf(1).

#### At larger values of n, you can also use the Normal distribution to approximate the binomial distribution.

In [6]:
import scipy.stats as scs

In [20]:
binom = scs.binom(100, 0.04)

In [21]:
binom.cdf(1)

0.08716331668738958

In [22]:
norm = scs.norm(4, np.sqrt(4*.96))

In [23]:
norm.cdf(1)

0.062893212319469996

If 70% of Facebook users on iOS use Instagram, but only 35% of Facebook users on Android use Instagram, how would you investigate the discrepancy?

How do you measure newsfeed health?



How would you measure the health of Mentions, Facebook's app for celebrities? How can FB determine if it's worth it to keep using it? If a celebrity starts to use Mentions and begins interacting with their fans more, what part of the increase can be attributed to a celebrity using Mentions, and what part is just a celebrity wanting to get more involved in fan engagement?

How would you measure the health of Mentions, Facebook's app for celebrities? How can FB determine if it's worth it to keep using it? If a celebrity starts to use Mentions and begins interacting with their fans more, what part of the increase can be attributed to a celebrity using Mentions, and what part is just a celebrity wanting to get more involved in fan engagement?

Given a specific product, come up with some potential improvements and design a series of experiments for testing/implementing these changes.

 A product manager is considering replacing an ad unit with a 'people you may know' unit on the news feed. What's your advice?